In [1]:
# Import modules.
import datetime
import numpy as np
import random

from geopy.distance import vincenty

from pyevolve import Crossovers
from pyevolve import Consts
from pyevolve import G1DList
from pyevolve import GSimpleGA
from pyevolve import Statistics

In [2]:
# Fitness function.
# It simply calculates the distance in kilometers of the current path.
def fitness(chromosome):
    score = 0
    for i in range(len(chromosome)):
        j = (i + 1) % len(chromosome)
        score += matrix[chromosome[i], chromosome[j]]
    return score

In [3]:
# Callback called each time for each generation.
def tsp_callback(ga_engine):
    stats.append(ga.getStatistics().asTuple())
    return False

In [4]:
# Initializator for the 1DList.
# Simply creates a list containing the cities indexes and shuffle it.
def list_initializator(genome, **args):
    cities = [i for i in range(len(coords))]
    random.shuffle(cities)
    genome.setInternalList(cities)

In [5]:
# Save the GA statistics in `filename`.
def save_stats(filename, stats):
    header = 'rawMax;rawMin;rawAve;rawDev;rawVar;fitMax;fitMin;fitAve'
    np.savetxt(filename, stats, '%.2f', delimiter=';', header=header, comments='')
    print('Saved stats into ' + filename)

In [6]:
# GPS position of each city.
LAT = [16.47, 16.47, 20.09, 22.39, 25.23, 22.00, 20.47, 17.20, 16.30, 14.05, 16.53, 21.52, 19.41, 20.09]
LON = [96.10, 94.44, 92.54, 93.37, 97.24, 96.05, 97.02, 96.29, 97.38, 98.12, 97.38, 95.59, 97.13, 94.55]

# Create a 1D list of tuples with latitude and longitude for each city.
coords = []
for i in range(len(LAT)):
    coords.append((LAT[i], LON[i]))

# Create a distance matrix between each city.
matrix = {}
for i, city1 in enumerate(coords):
    for j, city2 in enumerate(coords):
        dist = vincenty(city1, city2).km
        matrix[i, j] = dist
        
# Constants.
POPULATION_SIZE = 100
MUTATION_RATE   = 0.01
CROSSOVER_RATE  = 0.9
GENERATIONS_NB  = 300
CROSSOVER_FUNC  = Crossovers.G1DListCrossoverEdge

# Array for collecting statistics.
stats = []

# The chromosome is represented by a 1DList which contains the cities indexes [0,13]
genome = G1DList.G1DList(len(coords))

# Set initializator function.
genome.initializator.set(list_initializator)

# Set fitness function.
genome.evaluator.set(fitness)

# Set crossover function.
genome.crossover.set(CROSSOVER_FUNC)

# GA initialization.
ga = GSimpleGA.GSimpleGA(genome)
ga.setPopulationSize(POPULATION_SIZE)
ga.setMutationRate(MUTATION_RATE)
ga.setCrossoverRate(CROSSOVER_RATE)
# Elitism is important for TSP.
ga.setElitism(True)
# We aim for minimization.
ga.setMinimax(Consts.minimaxType["minimize"])

# Number of generations
ga.setGenerations(GENERATIONS_NB)

# In case we want to monitor the evolution process
# execute the function current_best every generation
ga.stepCallback.set(tsp_callback)

ga.evolve(freq_stats=50)

# Final best solution
print(ga.bestIndividual())

# Save statistics.
if CROSSOVER_FUNC == Crossovers.G1DListCrossoverEdge:
    crossover_name = 'edge'
else:
    crossover_name = 'perso'
    
now = datetime.datetime.now()

save_stats('stats/' +
               crossover_name +
               '_pop' + str(POPULATION_SIZE) +
               '_gen' + str(GENERATIONS_NB) +
               '_' + now.strftime('%d%m%Y-%H%M') +
               '.csv',
           stats)

# Best path found: [1, 13, 2, 3, 4, 5, 11, 6, 12, 7, 10, 8, 9, 0]
# For crossovers inspiration: https://www.hindawi.com/journals/cin/2017/7430125/

Gen. 0 (0.00%): Max/Min/Avg Fitness(Raw) [8108.93(8156.41)/5119.59(5062.05)/6757.44(6757.44)]
Gen. 50 (16.67%): Max/Min/Avg Fitness(Raw) [5085.34(6201.56)/3911.84(3482.56)/4237.79(4237.79)]
Gen. 100 (33.33%): Max/Min/Avg Fitness(Raw) [5009.11(5842.42)/3797.51(3421.46)/4174.26(4174.26)]
Gen. 150 (50.00%): Max/Min/Avg Fitness(Raw) [5041.78(6005.27)/3803.31(3346.76)/4201.48(4201.48)]
Gen. 200 (66.67%): Max/Min/Avg Fitness(Raw) [5246.69(6349.54)/3918.73(3346.76)/4372.24(4372.24)]
Gen. 250 (83.33%): Max/Min/Avg Fitness(Raw) [5580.72(6897.16)/4110.78(3346.76)/4650.60(4650.60)]
Gen. 300 (100.00%): Max/Min/Avg Fitness(Raw) [5044.00(6405.38)/3876.32(3346.76)/4203.33(4203.33)]
Total time elapsed: 2.384 seconds.
- GenomeBase
	Score:			 3346.761974
	Fitness:		 3876.323350

	Params:		 {}

	Slot [Evaluator] (Count: 1)
		Name: fitness - Weight: 0.50
	Slot [Initializator] (Count: 1)
		Name: list_initializator - Weight: 0.50
	Slot [Mutator] (Count: 1)
		Name: G1DListMutatorSwap - Weight: 0.50
		Doc:  T